In [ ]:
import pandas as pd
import fundingutils
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
ROUND_SELECTION = 'Tech' # EDIT THIS to 'Events' or 'Tech'

round_options = {'Tech': '0xd875Fa07bEdcE182377ee54488f08F017cb163D4', 'Events': '0xf89aad3fad6c3e79ffa3ccc835620fcc7e55f919'}

rounds = pd.read_csv('data/rounds_summary.csv')
rounds.drop('Unnamed: 0', axis=1, inplace=True)
rounds = rounds[rounds['round_id'].str.lower() == round_options[ROUND_SELECTION].lower()]
round_name = rounds['round_name'].astype(str).values[0] if 'round_name' else None
matching_cap_amount = rounds['matching_cap_amount'].astype(float).values[0] if 'matching_cap_amount' in rounds and not pd.isnull(rounds['matching_cap_amount'].values[0]) else 'No Cap'
matching_amount = rounds['matching_funds_available'].astype(float).values[0] if 'matching_funds_available' in rounds else 0
min_donation_threshold_amount = rounds['min_donation_threshold_amount'].astype(float).values[0] if 'min_donation_threshold_amount' in rounds and not pd.isnull(rounds['min_donation_threshold_amount'].values[0]) else 0.0

print(f"Round Name: {round_name}")
print(f"Matching Cap Amount: {matching_cap_amount} ETH")
print(f"Matching Amount: {matching_amount} ETH")
print(f"Minimum Donation Threshold Amount: ${min_donation_threshold_amount}")

In [ ]:
def allowlist_poap_holders(df):
    poap_holders = pd.read_csv('data/poap_combined.csv')
    poap_holders = poap_holders['address'].str.lower().tolist()
    df['is_allowlisted'] = df['voter'].apply(lambda x: True if x in poap_holders else False)
    if 'original_score' not in df.columns:
        df['original_score'] = df['score']
    else:
        df['score'] = df['original_score']
    df['score'] = df.apply(lambda row: 25 if row['original_score'] < 25 and row['is_allowlisted'] else row['score'], axis=1)
    return df


votes = pd.read_csv(f'data/{ROUND_SELECTION}_votes.csv')
votes = allowlist_poap_holders(votes)
poap_votes = votes[votes['is_allowlisted'] == True].copy()

voters = votes['voter'].nunique()
crowdfunded = votes['amountUSD'].sum()
poap_voters = poap_votes['voter'].nunique()
crowdfunded_poap = poap_votes['amountUSD'].sum()
percentage_crowdfunded_poap = (crowdfunded_poap/crowdfunded) * 100

print(f"Number of Voters: {voters}")
print(f"Amount Crowdfunded: ${crowdfunded:,.2f}")
print(f"Number of Voters holding a ZuPass POAP: {poap_voters}")
print(f"Amount Crowdfunded by POAP Holders: ${crowdfunded_poap:,.2f}")
print(f"Percentage of crowdfund from POAP holders: {percentage_crowdfunded_poap:,.2f}%")

In [ ]:
eligible_voters = votes[votes['score'] > 15]['voter'].nunique()
eligible_voters_percentage = (eligible_voters / voters) * 100
poap_voters_under_25 = poap_votes[poap_votes['original_score'] < 25]['voter'].nunique()
poap_voters_under_25_percentage = (poap_voters_under_25/poap_voters) * 100

print(f"Eligible Voters: {eligible_voters}")
print(f"Eligible Voters Percentage: {eligible_voters_percentage:,.2f}%")
print(f"POAP Voters with Score Under 25: {poap_voters_under_25}")
print(f"POAP Voters Under 25 Percentage: {poap_voters_under_25_percentage:,.2f}%")

dust_donations = len(votes[votes['amountUSD'] < 1])
donations_total = len(votes)
dust_donations_percentage = (dust_donations / donations_total) * 100

print("---")
print(f"Dust Donations: {dust_donations}")
print(f"Total Donations: {donations_total}")
print(f"Dust Donations Percentage: {dust_donations_percentage:,.2f}%")

total_donation_per_voter = votes.groupby('voter')['amountUSD'].sum()
total_donation_per_poap_voter = poap_votes.groupby('voter')['amountUSD'].sum()
average_donation_per_voter = total_donation_per_voter.mean()
average_donation_per_poap_voter = total_donation_per_poap_voter.mean()
poap_voters_donation_multiplier = round(average_donation_per_poap_voter / average_donation_per_voter, 2)
median_donation_per_voter = total_donation_per_voter.median()
median_donation_per_poap_voter = total_donation_per_poap_voter.median()

print("---")
print(f"Average Donation per Voter: ${average_donation_per_voter:,.2f}")
print(f"Average Donation per POAP Voter: ${average_donation_per_poap_voter:,.2f}")
print(f"POAP Voters Donate {poap_voters_donation_multiplier}x More on Average")
print(f"Median Donation per Voter: ${median_donation_per_voter:,.2f}")
print(f"Median Donation per POAP Voter: ${median_donation_per_poap_voter:,.2f}")


In [ ]:
# Define strategies
strategies = ['qf', 'pairwise', 'COCM']

# Prepare votes data
votes_prep = fundingutils.prep_donations_data(votes, min_donation_threshold_amount, 15)
votes_matrix = fundingutils.pivot_votes(votes_prep)

# Prepare POAP votes data
poap_votes_prep = fundingutils.prep_donations_data(poap_votes, min_donation_threshold_amount, 15)
poap_votes_matrix = fundingutils.pivot_votes(poap_votes_prep)

# Initialize matching_df
matching_df = pd.DataFrame()

# Iterate over strategies
for strategy in strategies:
    # Get QF matching for votes and POAP votes
    votes_qf_matching = fundingutils.get_qf_matching(strategy, votes_matrix, matching_cap_amount, matching_amount, cluster_df = None if strategy == 'qf' else votes_matrix)
    poap_votes_qf_matching = fundingutils.get_qf_matching(strategy, poap_votes_matrix, matching_cap_amount, matching_amount, cluster_df = None if strategy == 'qf' else poap_votes_matrix)
    
    # Rename columns
    votes_qf_matching = votes_qf_matching.rename(columns={'project_name': 'Project', 'matching_amount': 'Matching Amount'})
    poap_votes_qf_matching = poap_votes_qf_matching.rename(columns={'project_name': 'Project', 'matching_amount': 'Matching Amount'})
    
    # Add strategy column
    votes_qf_matching['Strategy'] = strategy.capitalize() if strategy != 'COCM' else 'Cluster Match'
    poap_votes_qf_matching['Strategy'] = f'POAP {strategy.capitalize()}' if strategy != 'COCM' else 'POAP Cluster Match'
    
    # Concatenate dataframes
    matching_df = pd.concat([matching_df, votes_qf_matching, poap_votes_qf_matching])


# Group by 'Project' and 'Strategy'
grouped_df = matching_df.groupby(['Project', 'Strategy'])['Matching Amount'].mean().unstack()
# Calculate the percentage difference
grouped_df['Pct_Diff'] = (grouped_df['POAP Qf'] - grouped_df['Qf']) / grouped_df['Qf'] * 100

# Reset the index
grouped_df = grouped_df.reset_index()
# Merge the dataframes on 'Project'
matching_df = pd.merge(matching_df, grouped_df[['Project', 'Pct_Diff']], on='Project', how='left')
# Sort dataframe
matching_df = matching_df.sort_values(by='Pct_Diff', ascending=False)
matching_df['Project_Short'] = matching_df['Project'].apply(lambda x: x[:15] + '...' if len(x) > 20 else x)

# OPTIONAL: save the dataframe to a csv file
# matching_df.to_csv(f'data/zuzalu_{ROUND_SELECTION}_matching_df.csv', index=False)


In [ ]:
# OPTIONAL: read the matching dataframe from a csv file
# matching_df = pd.read_csv(f'data/zuzalu_{ROUND_SELECTION}_matching_df.csv')
# print(f'Read file succesfully: data/zuzalu_{ROUND_SELECTION}_matching_df.csv')
# print('Shape:' + str(matching_df.shape))

In [ ]:
# SET GRAPH FORMATTING
strategies = ['Qf', 'POAP Qf', 'Pairwise',  'POAP Pairwise','Cluster Match', 'POAP Cluster Match']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#bcbd22', '#7f7f7f']
names = ['QF - All',  'QF - POAP', 'Pairwise - All', 'Pairwise - POAP','Cluster - All', 'Cluster - POAP']

font = dict(
            family="Arial, sans-serif",
            size=14,
            color="DarkSlateGrey"
        )
xaxis = dict(
            title_standoff=20,
            automargin=True,
            tickfont=dict(size=10),
            tickangle=45
        )
yaxis = dict(
            title_standoff=20,
            automargin=True 
        )


In [ ]:
def generate_matching_bargraph(df, group):
    # Define the strategies for the x-axis
    if group == 'POAP':
        df = df[df['Strategy'].str.contains('POAP')]
    elif group == 'non POAP':
        df = df[~df['Strategy'].str.contains('POAP')]
    elif group == 'Qf':
        df = df[df['Strategy'].str.contains('Qf')]

    # Create a figure
    fig = go.Figure()
    # Add a bar plot for each strategy with custom colors
    for strategy, color, name in zip(strategies, colors, names):
        strategy_df = df[df['Strategy'] == strategy]
        trace = go.Bar(x=strategy_df['Project_Short'], 
                       y=strategy_df['Matching Amount'], 
                       name=name, 
                       hovertext=strategy_df['Project'], 
                       marker=dict(color=color))
        fig.add_trace(trace)
    fig.update_layout(
        title={
            'text': f'Zuzalu {ROUND_SELECTION}: QF Matching Comparison',
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=24, color='DarkSlateGrey')},
        xaxis_title='Project',
        yaxis_title='Match Amount (ETH)',
        height=600,
        font=font,
        xaxis=xaxis,
        yaxis=yaxis,
        yaxis_range=([0, 25]),
        legend=dict(
            yanchor="top",
            y=1.0,
            xanchor="right",
            x=1.0,
            bgcolor='rgba(255, 255, 255, 0.5)'
        )
    )

    # Display the plot
    ranked_projects = {}
    for strategy in df['Strategy'].unique():
        sorted_projects = df[df['Strategy'] == strategy].sort_values(by='Matching Amount', ascending=False)['Project_Short']
        ranked_projects[strategy] = sorted_projects.reset_index(drop=True)
    df_pivot = pd.DataFrame(ranked_projects)
    return df_pivot, fig

# Use the function to filter data and plot graph
group = 'POAP'  # Options: POAP, non POAP, Qf
df_pivot, fig = generate_matching_bargraph(matching_df, group)
fig.show()
df_pivot.head(7)


In [ ]:
fig = px.scatter(matching_df.sort_values(by='Pct_Diff', ascending=True), y='Project_Short', x='Pct_Diff', hover_data=['Project'], labels={'Pct_Diff':'Percentage Difference', 'Project_Short':'Project'}, title=f'{ROUND_SELECTION} Round: % Diff in QF Matching with ZuPass POAP',height=720,  color_discrete_sequence=["black",'#1f77b4', '#ff7f0e'])
# Add short lines to each row
for _, row in matching_df.iterrows():
    line_color = "#ff7f0e" if row['Pct_Diff'] > 0 else "#1f77b4"
    fig.add_shape(type="line",
                  x0=row['Pct_Diff'], y0=row['Project_Short'], 
                  x1=row['Pct_Diff'] - ( row['Pct_Diff']), y1=row['Project_Short'],
                  line=dict(color=line_color,width=2))
fig.update_layout(
    title={
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=xaxis,
    yaxis=yaxis,

)


In [ ]:
# STRATEGY BOX PLOTS
fig = go.Figure()
for strategy, color, name in zip(strategies, colors, names):
    fig.add_trace(go.Box(y=matching_df[matching_df['Strategy'] == strategy]['Matching Amount'],
                         name=name,
                         marker_color=color))
fig.update_layout(
    title={
        'text': f'Zuzalu {ROUND_SELECTION}: Distribution of Match Amounts by Strategy',
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='DarkSlateGrey')
    },
    xaxis_title='Matching Strategy',
    yaxis_title='Match Amount (ETH)',
    xaxis=xaxis,
    yaxis=yaxis,
    height=600,
    font=font,
    #yaxis_type='log',  # Can use Logarithmic scale for better visualization with outliers like Edge City
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=1.25
    ),
    boxmode='group'  
)
fig.show()


In [ ]:
# HEATMAP - PROB WONT USE

fig = go.Figure(data=go.Heatmap(
    z=matching_df['Matching Amount'],  
    x=matching_df['Strategy'],  
    y=matching_df['Project_Short'].tolist(),  
    colorscale='Viridis',  
    colorbar=dict(title='Match Amount (ETH)'),
    hoverongaps=False  
))

fig.update_layout(
    title={
        'text': 'Heatmap of Match Amounts by Project and Strategy',
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='DarkSlateGrey')
    },
    xaxis_title='Matching Strategy',
    yaxis_title='Project',
    xaxis=xaxis,
    yaxis=yaxis,
    height=800,  
    font=font,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=1
    )
)
fig.show()

In [ ]:
# CREATE PIVOT DF FOR EXPORT
# Allows for comparing project outcomes by strategy and viewing project-level crowdfunding numbers
pivot_df = matching_df.pivot(index='Project', columns='Strategy', values='Matching Amount')
pivot_df['crowdfunded_amountUSD'] = votes.groupby('project_name')['amountUSD'].sum()
pivot_df['eligible_voters_count'] = votes[votes['score'] >= 15].groupby('project_name')['voter'].nunique()
pivot_df['poap_voters_count'] = poap_votes.groupby('project_name')['voter'].nunique()
pivot_df.head(10)


In [ ]:
pivot_df.to_csv(f'data/zuzalu_{ROUND_SELECTION}_pivot_df.csv')
print(f'Saved file succesfully: data/zuzalu_{ROUND_SELECTION}_pivot_df.csv')